# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 759 ms, sys: 263 ms, total: 1.02 s
Wall time: 576 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_180_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_180_6561"
random_state = 11


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180980 entries, 0 to 1180979
Columns: 188 entries, id to feat179
dtypes: float64(183), int32(3), object(2)
memory usage: 1.6+ GB
CPU times: user 16.9 s, sys: 869 ms, total: 17.8 s
Wall time: 17.8 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  180
n_hamiltonians:  6561
n_ts:  2
CPU times: user 116 µs, sys: 7 µs, total: 123 µs
Wall time: 80.3 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.5450388660265203
% val:  0.0
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  3576
number of val hamiltonians:  0
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:53<1:27:55, 53.29s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [01:46<1:26:49, 53.15s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:25:58<00:00, 50.99s/it]

running experiments: 100%|██████████| 100/100 [1:25:58<00:00, 51.58s/it]

CPU times: user 9h 27min 46s, sys: 19 s, total: 9h 28min 5s
Wall time: 1h 25min 58s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.018047,0.037394,...,0.029254,0.015857,0.004441,0.027159,-0.041982,-0.021236,0.005203,-0.024027,0.037394,0.018047
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.084825,-0.026222,...,-0.104377,-0.066035,0.055706,-0.051816,0.080410,0.116916,-0.119805,-0.040657,0.026222,-0.084825
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.043223,0.005021,...,0.026354,-0.048356,0.035233,0.067210,-0.054791,-0.024377,0.024692,-0.018139,-0.005021,0.043223
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.009349,-0.031973,...,0.043557,0.051909,-0.042604,-0.009129,0.022361,-0.026100,0.011385,0.036684,-0.031973,-0.009349
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.028827,0.057672,...,0.053523,0.039631,-0.002767,0.038861,-0.065675,-0.039973,0.010875,-0.033361,0.057672,0.028827


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 169 ms, sys: 0 ns, total: 169 ms
Wall time: 168 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
347621,1931,H_-0.85_1.4_0.00045794.dat,-0.85,1.40,0.000458,0,0,val,0.085505,0.075016,...,0.040330,0.043438,-0.102703,-0.104778,0.094830,0.102161,-0.089174,-0.061077,-0.075016,-0.085505
579075,3217,H_-0.05_0.9_0.000856259.dat,-0.05,0.90,0.000856,0,0,val,-0.036225,0.011702,...,0.101670,0.075648,0.102816,0.101272,0.083330,0.104283,0.030806,0.078278,-0.011702,0.036225
855035,4750,H_0.9_0.6_-1.00398.dat,0.90,0.60,-1.003977,999,1,test,-0.037818,-0.014182,...,0.057001,0.035591,0.098905,0.087386,0.101621,0.104981,0.064277,0.085400,0.014182,0.037818
1104771,6137,H_1.75_1.1_-1.00492.dat,1.75,1.10,-1.004923,999,0,test,0.021624,0.085579,...,-0.064824,0.029116,-0.003177,-0.074530,0.024997,-0.028326,0.052204,0.051453,-0.085579,-0.021624
361710,2009,H_-0.8_1.25_0.000370351.dat,-0.80,1.25,0.000370,0,0,val,0.030750,0.028063,...,-0.085906,0.060330,-0.075148,0.075738,-0.063348,0.088881,-0.007318,0.069265,0.028063,0.030750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284001,1577,H_-1.05_-0.1_-0.00256881.dat,-1.05,-0.10,-0.002569,0,0,train,-0.008838,0.049137,...,0.096159,0.069307,-0.092789,-0.067128,0.081648,0.039407,-0.059641,-0.019115,0.049137,-0.008838
820895,4560,H_0.8_-0.8_1.00122.dat,0.80,-0.80,1.001220,1,1,val,0.025832,0.005998,...,0.006241,-0.103082,0.029149,0.075599,-0.015889,-0.002646,-0.013631,-0.048204,0.005998,0.025832
923138,5128,H_1.15_-0.75_1.00077.dat,1.15,-0.75,1.000771,1,1,train,0.047174,-0.055828,...,0.059118,-0.026751,-0.089668,0.070861,0.103586,-0.100879,-0.100851,0.092464,0.055828,-0.047174
842778,4682,H_0.85_1.25_2.00637.dat,0.85,1.25,2.006372,999,0,test,-0.065293,-0.065142,...,0.004364,0.023420,-0.021718,-0.008262,-0.064823,-0.032439,-0.054097,-0.051424,-0.065142,-0.065293


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.27251943301326015
% val:  0.27251943301326015
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  1788
number of val hamiltonians:  1788
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [3284, 930, 78, 5116, 282, 1448, 4321, 2058, 6168, 3163, 1112, 3375, 3841, 2862, 5777, 2928, 5934, 2797, 5048, 3921, 3187, 3254, 1835, 3329, 3042, 1731, 2802, 3105, 3207, 3493, 2317, 6080, 5448, 2729, 2818, 1490, 3736, 1854, 3754, 3645, 645, 2146, 3023, 5451, 4538, 5124, 2174, 4315, 809, 1282, 4259, 637, 1676, 4078, 3982, 4805, 4642, 4088, 723, 2781, 2640, 1367, 953, 646, 4465, 862, 1181, 2966, 3335, 1414, 1284, 3614, 1004, 621, 1105, 2842, 6009, 139, 4334, 5292, 3490, 1357, 2805, 4161, 2631, 641, 3523, 1600, 3663, 3262, 1918, 629, 2535, 242, 1987, 3991, 684, 2609, 3760, 3833, 2735, 6336, 3565, 1106, 1115, 2886, 6090, 1288, 5207, 2961, 3836, 3359, 4628, 4400, 2714, 4957, 230, 3247, 1506, 612, 383, 4946, 3004, 6002, 800,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 101 µs, sys: 1e+03 ns, total: 102 µs
Wall time: 58.7 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.550000,0.450000,999,-1
1,-2.0,-1.95,test,0.505556,0.494444,999,-1
2,-2.0,-1.90,test,0.505556,0.494444,999,-1
3,-2.0,-1.85,test,0.561111,0.438889,999,-1
4,-2.0,-1.80,test,0.577778,0.422222,999,-1
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.677778,0.322222,999,-1
6557,2.0,1.85,test,0.677778,0.322222,999,-1
6558,2.0,1.90,test,0.738889,0.261111,999,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1180980


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,0,test
2,0,999,1,test
3,0,999,1,test
4,0,999,1,test
...,...,...,...,...
1180975,6560,999,0,test
1180976,6560,999,0,test
1180977,6560,999,0,test
1180978,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9998819289087746,
 'eigenvector_val': 0.9900478498632861,
 'eigenvector_test': 0.8118464052287582,
 'hamiltonian_train': 0.0,
 'hamiltonian_val': 0.0,
 'hamiltonian_test': 0.014705882352941176}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}